# # RFM ile Müşteri Segmentasyonu (Customer Segmentation with RFM)

1. İş Problemi (Business Problem)
2. Veriyi Anlama (Data Understanding)
3. Veri Hazırlama (Data Preparation)
4. RFM Metriklerinin Hesaplanması (Calculating RFM Metrics)
5. RFM Skorlarının Hesaplanması (Calculating RFM Scores)
6. RFM Segmentlerinin Oluşturulması ve Analiz Edilmesi (Creating & Analysing RFM Segments)
7. Tüm Sürecin Fonksiyonlaştırılması


1. İş Problemi (Business Problem) :
Bir e-ticaret şirketi müşterilerini segmentlere ayırıp bu segmentlere göre
pazarlama stratejileri belirlemek istiyor.

Veri Seti Hikayesi
https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

Online Retail II isimli veri seti İngiltere merkezli online bir satış mağazasının
01/12/2009 - 09/12/2011 tarihleri arasındaki satışlarını içeriyor.
Bu çalışmamda 2009-2010 tarihleri arasındaki satışların verilerini inceleyeceğim.

Değişkenler

InvoiceNo:    Fatura numarası. Her işleme yani faturaya ait eşsiz numara. C ile başlıyorsa iptal edilen işlem.

StockCode:    Ürün kodu. Her bir ürün için eşsiz numara.

Description:  Ürün ismi

Quantity:     Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.

InvoiceDate:  Fatura tarihi ve zamanı.

UnitPrice:    Ürün fiyatı (Sterlin cinsinden)

CustomerID:   Eşsiz müşteri numarası

Country:      Ülke ismi. Müşterinin yaşadığı ülke.




#  2. Veriyi Anlama (Data Understanding)

In [9]:
import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', None) 
#yazdırma işleminde bütün sutunlar gözüksün

# pd.set_option('display.max_rows', None) #bütün satırları gör

pd.set_option('display.float_format', lambda x: '%.3f' % x) 
#sayısal değişkenlerin virgülden sonra kaç basamağını göstermeliyim.

df_ = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2009-2010") 
#sheet name: veriseti 2 syafa biz ilk syafayı oku dedik.

df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom


In [10]:
df.shape
#boyut bilgisi

(525461, 8)

In [15]:
df.isnull().sum() 
#Eksik değerlere baktık,var olan eksik değerler müşteri özelinde segmentasyon çalışması yapacağımız için 
#bu ölçülebilirlik değeri taşımadığından  bunları sonra sileceğiz.
#Description da ki eksik veriler gözardı edilebilir, fakat customer ıd için bunu söylemeyiz çünkü eksik verilerin 
#çok olması sonuçlarıda saptıracaktır.


Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

# 3. Veri Hazırlama (Data Preparation)

In [17]:
df.dropna(inplace=True)
#dropna ifadesi eksik değerleri silmek için kullanılır.
#inplace argümanı yeniden atama işlemine gerek kalmaksızın değişikliğin kalıcı olmasını sağlar.

In [32]:
df.shape
#yaklaşık 100 bin gözlem verisetimizin eksek değerler açısından temizlenmiş oldu.

(407695, 9)

In [33]:
#invoice da başında C olan ifadeler vardı iade işlemleri temsil ediyordu bu iadeler veri setini özetlediğimizde
#bize negatif değerler gösterecektir. Describe ile sayısal değerlerin dağılımına bakalım.

df.describe().T

# Describe,NaN değerleri hariç olmak üzere bir veri kümesinin dağılımının merkezi eğilimini, 
# dağılımını ve şeklini özetleyen tanımlayıcı istatistikler oluşturmak için kullanılır.
# count - Boş olmayan değerlerin sayısı.
# mean - Ortalama (ortalama) değer.
# std - Standart sapma.
# min - minimum değer.
# %25 - %25 yüzdelik dilimi.
# %50 - %50 yüzdelik dilim.
# %75 - %75 yüzdelik dilimi.
# maks - maksimum değer.
# >>Sonuç olarak;
# Oluşan tabloda sayısal değerlere  baktığımızda Quantity’de negatif değer görmememiz gerekir. 
# Price negatif değerleri ise iade olan işlemler olabilir. 
# İade olan işlemlerin faturaları C ile başlıyordu. Bu yüzden C harfini içeren faturaları siliyoruz.

,count,mean,std,min,25%,50%,75%,max
Quantity,407695.000,13.587,96.842,1.000,2.000,5.000,12.000,19152.000
Price,407695.000,3.294,34.757,0.000,1.250,1.950,3.750,10953.500
Customer ID,407695.000,15368.504,1679.796,12346.000,13997.000,15321.000,16812.000,18287.000
Total_Price,407695.000,21.663,77.147,0.000,4.950,11.900,19.500,15818.400


In [34]:
df = df[~df["Invoice"].apply(str).str.contains("C", na=False)] 
#iade edilmiş değerler eksi(-) sonuç getiriyordu bu değerler veri setinin yapısını bozmakta
#bunun için C haric diğer verileri getir dedik.

# 4. RFM Metriklerinin Hesaplanması (Calculating RFM Metrics)
RFM Analizi Müşteri Segmentasyonu İçin Kullanılan Bir Tekniktir.
Müşterilerin Satın Alma Alışkanlıkları Üzerinden Gruplara Ayrılması Ve 
Bu Gruplar Özelinde Stratejiler Geliştirilebilmesini Sağlar.
CRM Çalışmaları İçin Birçok Başlıkta Veriye Dayalı Aksiyon Alma İmkanı Sağlar.

Rfm Metrikleri

-Recency(Yenilik):Müşterinin Yenilik Yada Bizden En Son Ne Zaman Alışveriş Yaptığı Durumunu İfade Etmektedir.
Örneğin Bir Müşterinin Recency 1 Diğerinin 10 İse 1 Olan Bizim İçin Daha İyidir.
Henüz 1 Gün Önce Alışveriş Yapmış Demektir.Günlük Bazda Değerlendirme Yaptık.
Recency Müşterinin Sıcaklığını Yeniliğini İfade Eder.

-Frequency(Sıklık):Müşterinin Yaptığı Toplam Alışveriş Sayısıdır.
Diğer İfadeyle İşlem Sıklığıdır.

-Monetary(Parasal Değer):Müşterilerin Bize Bıraktığı Parasal Değeri İfade Eder.

!!CRM ve RFM ile ilgili detaylı açıklamaları klasör içerisinde pdf dosyasını paylaşmış olacağım. 
Şimdi çalışmamıza geri dönelim.



In [40]:
df["InvoiceDate"].max()
#veriseti içerisindeki en son tarih
#biz o yıllarda yaşamadığımız için ilgili hesaplamaların yapılması için şöyle bir yol izlememiz gerekmektedir.
#analizin yapıldığı günü tanımlamamız gerekmektedir. Data içerisindeki en son tarihten 2 gün sonra
#bu analizi yaptığımızı varsayalaım. Çalışmanın başında import ettiğimiz datetime kütüphanesinden
#today_date adlı tarih değişkeni oluşturmak için yardım istiyoruz.


Timestamp('2010-12-09 20:01:00')

In [36]:
today_date = dt.datetime(2010, 12, 11) 
#analizin yapıldığı varsaydığımız tarihi  oluşturduk.
#zaman formunda bir değişken
today_date

datetime.datetime(2010, 12, 11, 0, 0)

In [37]:
type(today_date)

datetime.datetime

In [39]:
#veri seti içerinde ürün ve fiyat bilgi mevcut fakat toplam fiyat değişkenimiz yok bunu biz veri setimize ekleyelim.
df['Total_Price']=df['Quantity']*df['Price']
df['Total_Price']

0         83.400
1         81.000
2         81.000
3        100.800
4         30.000
           ...  
525456     5.900
525457     3.750
525458     3.750
525459     7.500
525460     3.900
Name: Total_Price, Length: 407695, dtype: float64

In [43]:
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                     'Invoice': lambda Invoice: Invoice.nunique(),
                                     'Total_Price': lambda TotalPrice: TotalPrice.sum()})
#foksiyonun okunuşu:
#Dataframe 'Customer ıd' ye göre groupby değişkenine al veri setindeki diğer değişkenlere göre senin çeşitli  aggregation
#yani hesaplamalar yapman lazım.
#aggregation fonksiyonu sözlük yapısıyla, key kısmında değişkenleri ve value kısmında da bu değişkenlere
#uygulamak isteyeceğin fonksiyonu ifade edersen ben bunları ilgili fonksiyonlara uygularım  der.

#müşterilere göre groupby aldıktan sonra  fonksiyon şunun yapılmasını sağlayacak.

#(recency):bugünün tarihinden - müşterinin son satın alma tarihini çıkartacak bunu gün cinsinden verecek.
#(frequency):kullanıcılara göre groupbya aldıktan sonra invoiceların eşsiz değerlerini al, bak bakalım kaç tane fatura var.
#(monetary): müşterrilerin bize bıraktığı toplam para.
rfm.head()

,InvoiceDate,Invoice,Total_Price
Customer ID,,,
12346.000,165,11,372.860
12347.000,3,2,1323.320
12348.000,74,1,222.160
12349.000,43,3,2671.140
12351.000,11,1,300.930


In [47]:
rfm.columns = ['recency', 'frequency', 'monetary'] 
#kolon isimlerini değiştirdik.
rfm.head()

,recency,frequency,monetary
Customer ID,,,
12346.000,165,11,372.860
12347.000,3,2,1323.320
12348.000,74,1,222.160
12349.000,43,3,2671.140
12351.000,11,1,300.930


In [50]:
rfm.describe().T
#monetary değerinde sıfır olması istediğimiz bir durum olmayabilir bunu veri setinden çıkartıyoruz.

,count,mean,std,min,25%,50%,75%,max
recency,4314.000,91.270,96.944,1.000,18.000,53.000,136.000,374.000
frequency,4314.000,4.454,8.169,1.000,1.000,2.000,5.000,205.000
monetary,4314.000,2047.289,8912.523,0.000,307.950,705.550,1722.802,349164.350


In [52]:
rfm = rfm[rfm["monetary"] > 0] 
#monetary değeri sıfırdan büyük olanları getirdik.
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4312.000,91.173,96.861,1.000,18.000,53.000,136.000,374.000
frequency,4312.000,4.456,8.170,1.000,1.000,2.000,5.000,205.000
monetary,4312.000,2048.238,8914.481,2.950,307.988,706.020,1723.142,349164.350


In [54]:
#müşteri ıd lere göre verimizi tekilleştirmiş olduk artık yeni veri setimizi oluşturduk. Bunlar rfm metrikleri biz bunları
#rfm skorlara dönüştürüp okunabilirliğini artıracağız.
rfm.shape

(4312, 3)

In [56]:
rfm.head()

,recency,frequency,monetary
Customer ID,,,
12346.000,165,11,372.860
12347.000,3,2,1323.320
12348.000,74,1,222.160
12349.000,43,3,2671.140
12351.000,11,1,300.930


# 5. RFM Skorlarının Hesaplanması (Calculating RFM Scores)


In [59]:
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
#qcut fonk. ile verilen değişkeni kaç parçaya bölmek istediğini söyle ve bölme işlemi sonrasında etiketleri söyle der.
#recency değerini küçükten büyüğe doğru sırala. sıcak müşteriye 5 puan uzak olan müşeriye 1 puan ver. 
#Yukarda receny değerini gün cinsinden hesaplamıştık receny değeri 1 ise yani 1 gün önce alışveriş yapmış 
#bunun recency_skoru 5.
#recency değeri 100 ise 100 gün önce alışveriş yapmış 
#bunun recency_skoru 1   böylece ölçülebilir hale geldi.
# 0-100, 0-20, 20-40, 40-60, 60-80, 80-100

rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
#ferakns değişkeninde ortaya çıkacak bir problemin çözümü olarakta  rank methodunu fırst olarak kullandık.
#Müşterinin yapmış olduğu alışveriş sıklığını yani işlem sıklığını hesapladık 
#bu değerlere artan şekilde etiket oluşturup atadık.

rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])
#monetary ile müşterinin yapmış olduğu toplam alışverişin parasal karşılığını 
#monetary_score ile artan şekilde etiket oluşturup atadık.
#küçük olana küçük etiketi ver büyük olana büyük etiketi ver.
#quantile q argümanı burda 5 tir.



In [61]:
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str))
#rfm skorlarına gitmek için r ve f değerleri yeterlidir monetary hesabını gözlemlemek için yaptık.


In [63]:
rfm

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12346.000,165,11,372.860,2,5,2,25
12347.000,3,2,1323.320,5,2,4,52
12348.000,74,1,222.160,2,1,1,21
12349.000,43,3,2671.140,3,3,5,33
12351.000,11,1,300.930,5,1,2,51
...,...,...,...,...,...,...,...
18283.000,18,6,641.770,4,5,3,45
18284.000,67,1,461.680,3,2,2,32
18285.000,296,1,427.000,1,2,2,12


In [65]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,407695.000,13.587,96.842,1.000,2.000,5.000,12.000,19152.000
Price,407695.000,3.294,34.757,0.000,1.250,1.950,3.750,10953.500
Customer ID,407695.000,15368.504,1679.796,12346.000,13997.000,15321.000,16812.000,18287.000
Total_Price,407695.000,21.663,77.147,0.000,4.950,11.900,19.500,15818.400


In [69]:
#En iyi 5 müşterimizi listeleyelim
rfm[rfm["RFM_SCORE"] == "55"].head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12415.000,11,7,19543.840,5,5,5,55
12431.000,9,13,4370.520,5,5,5,55
12471.000,10,49,20139.740,5,5,5,55
12472.000,5,13,11308.480,5,5,5,55
12474.000,14,13,5048.660,5,5,5,55


In [72]:
#En kötü 5 müşterimizi listeleyelim
rfm[rfm["RFM_SCORE"] == "11"].head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12355.000,203,1,488.210,1,1,2,11
12362.000,374,1,130.000,1,1,1,11
12366.000,269,1,500.240,1,1,2,11
12368.000,264,1,917.700,1,1,3,11
12378.000,198,1,1407.700,1,1,4,11


# 6. RFM Segmentlerinin Oluşturulması ve Analiz Edilmesi (Creating & Analysing RFM Segments)



In [76]:
# RFM isimlendirmesi
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}
#seg_map regex i ifade etmektedir. Bize şu sorguyu ve yapı yakalamayı sağlar.RFM_SCORE içerisinde 2 tane değerimiz var
#1. elemanında 1 ya da 2 ,2. elemanında 1 yada  2 görürsen yani keylerde ki ifadeleri görürsen eğer
#value lar da ki belirttiğimiz  isimlendirmeyi yap.
#köşeli parantezler bir elemanla ilgili ya da olasılıkları göz önünde bulundurulması gereken durumları ifade etmektedir.

rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True) 
#replace ile skorları birleştirdik
#rfm skolarını replace ile yukarıdaki key value değerleriyle değiştir.
rfm['segment'].head()

Customer ID
12346.000             cant_loose
12347.000    potential_loyalists
12348.000            hibernating
12349.000         need_attention
12351.000          new_customers
Name: segment, dtype: object

In [78]:
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])
#müşterileri segmentlere ayırdık
#metrikleri segmentlere göre groupby  al, ortalama ve toplam sayılarını getir.

recency       frequency       monetary      
                       mean count      mean count     mean count
segment                                                         
about_to_sleep       53.819   343     1.201   343  441.320   343
at_Risk             152.159   611     3.074   611 1188.878   611
cant_loose          124.117    77     9.117    77 4099.450    77
champions             7.119   663    12.554   663 6852.264   663
hibernating         213.886  1015     1.126  1015  403.978  1015
loyal_customers      36.287   742     6.830   742 2746.067   742
need_attention       53.266   207     2.449   207 1060.357   207
new_customers         8.580    50     1.000    50  386.199    50
potential_loyalists  18.793   517     2.017   517  729.511   517
promising            25.747    87     1.000    87  367.087    87

In [80]:
rfm[rfm["segment"] == "cant_loose"].head() 
#kaybedemeyeceğimiz kişilere 
#recency değeri 2 demek ki son zamanlarda bizden biraz uzaklaşmış
#frequency değeri 5 en yüksek frekansa sahip müşterilerimiz.

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.000,165,11,372.860,2,5,2,25,cant_loose
12380.000,101,7,6951.490,2,5,5,25,cant_loose
12482.000,212,29,23691.400,1,5,5,15,cant_loose
12510.000,95,7,4195.450,2,5,5,25,cant_loose
12891.000,94,8,509.500,2,5,3,25,cant_loose


In [83]:
rfm[rfm["segment"] == "champions"].head() 
#en iyi müşterilerimiz şampiyonlar grubu.

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12360.000,15,5,1590.830,5,4,4,54,champions
12395.000,7,4,2048.640,5,4,4,54,champions
12415.000,11,7,19543.840,5,5,5,55,champions
12429.000,1,5,3735.510,5,4,5,54,champions
12431.000,9,13,4370.520,5,5,5,55,champions


In [85]:
rfm[rfm["segment"] == "about_to_sleep"].head()
#uyumak üzere olan müşterilerimiz uzun zamandır etkileşimde kalamadığımız bu müşteri gruu için
#birşeyler yapmamız gerekebilir bunun içinde maliyet hesaplayıp karar verilmeli.


,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12353.000,44,1,317.760,3,1,2,31,about_to_sleep
12379.000,58,2,767.980,3,2,3,32,about_to_sleep
12387.000,51,1,143.940,3,1,1,31,about_to_sleep
12400.000,50,1,205.250,3,1,1,31,about_to_sleep
12411.000,45,1,346.900,3,1,2,31,about_to_sleep


In [87]:
rfm[rfm["segment"] == "new_customers"].head()
#yeni  müşterilerimiz.

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12351.000,11,1,300.930,5,1,2,51,new_customers
12385.000,15,1,1938.400,5,1,4,51,new_customers
12386.000,2,1,258.900,5,1,2,51,new_customers
12427.000,7,1,303.500,5,1,2,51,new_customers
12441.000,2,1,173.550,5,1,1,51,new_customers


In [91]:
rfm[rfm["segment"] == "cant_loose"].index
#kaybedemeyeceğimiz müşterilerin index bilgileri

Float64Index([12346.0, 12380.0, 12482.0, 12510.0, 12891.0, 12932.0, 13044.0,
              13313.0, 13680.0, 13782.0, 13799.0, 13856.0, 14025.0, 14063.0,
              14160.0, 14221.0, 14548.0, 14607.0, 14685.0, 14745.0, 15003.0,
              15013.0, 15015.0, 15125.0, 15141.0, 15222.0, 15306.0, 15321.0,
              15359.0, 15369.0, 15372.0, 15443.0, 15538.0, 15607.0, 15633.0,
              15722.0, 15751.0, 15754.0, 15768.0, 15911.0, 15912.0, 16027.0,
              16032.0, 16158.0, 16177.0, 16197.0, 16335.0, 16467.0, 16631.0,
              16742.0, 16743.0, 16875.0, 16986.0, 17021.0, 17032.0, 17092.0,
              17113.0, 17157.0, 17188.0, 17230.0, 17268.0, 17426.0, 17448.0,
              17454.0, 17512.0, 17578.0, 17602.0, 17651.0, 17940.0, 17969.0,
              17988.0, 18009.0, 18051.0, 18064.0, 18094.0, 18251.0, 18258.0],
             dtype='float64', name='Customer ID')

In [93]:
new_df = pd.DataFrame()
new_df["new_customer_id"] = rfm[rfm["segment"] == "new_customers"].index 
#yeni dataframe içerisine yeni müşterilerin ıd lerini attık.Bu bilgiler csv dosyasına aktarıp 
#ilgili departmana göndereceğiz.
new_df

,new_customer_id
0,12351.000
1,12385.000
2,12386.000
3,12427.000
4,12441.000
5,12538.000
6,12686.000
7,12738.000
8,12763.000
9,12767.000


In [96]:
new_df["new_customer_id"] = new_df["new_customer_id"].astype(int) #ondalık değerlerden kurtulduk.

In [98]:
new_df.to_csv("new_customers.csv") 
#istediğimiz verileri csv dosyası olarak dışarı çıkardık.
rfm.to_csv("rfm.csv")


# 7. Tüm Sürecin Fonksiyonlaştırılması

In [100]:

def create_rfm(dataframe, csv=False):

    # VERIYI HAZIRLAMA
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].apply(str).str.contains("C", na=False)]

    # RFM METRIKLERININ HESAPLANMASI
    today_date = dt.datetime(2011, 12, 11)
    rfm = dataframe.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                                'Invoice': lambda num: num.nunique(),
                                                "TotalPrice": lambda price: price.sum()})
    rfm.columns = ['recency', 'frequency', "monetary"]
    rfm = rfm[(rfm['monetary'] > 0)]

    # RFM SKORLARININ HESAPLANMASI
    rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
    rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
    rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

    # cltv_df skorları kategorik değere dönüştürülüp df'e eklendi
    rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
                        rfm['frequency_score'].astype(str))


    # SEGMENTLERIN ISIMLENDIRILMESI
    seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }

    rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)
    rfm = rfm[["recency", "frequency", "monetary", "segment"]]
    rfm.index = rfm.index.astype(int)

    if csv:
        rfm.to_csv("rfm.csv")

    return rfm

df = df_.copy()

rfm_new = create_rfm(df, csv=True)


# Özet: 
#Elimizdde bir veri seti vardı,fatura üzerinde oluşan veri setleri olarak düşünülebilir.
#Burada ki bütün odak satın alma işlemini raporlamaktır.
#Öncelikle bu veri setini kullanıcı özelinde tekilleştirmemiz gerekiyordu fakat bazı ön işlemler yaptık.
#Daha sonra rfm metriklerini hesapladık.Bu metrikleri skorlara çevirdik.
#Skorlar üzerinden segment isimlendirmelerini ve analizlerini gerçekleştirip analiz sonuçlarını bir csv dosyasına yazdık 
#ve bütün süreci fonksiyonlaştırıp işlemlerimizi tamamladık.
